## Day 22 - 3D sand block simulation... :panik:

In [41]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['1,0,1~1,2,1',
 '0,0,2~2,0,2',
 '0,2,3~2,2,3',
 '0,0,4~0,2,4',
 '2,0,5~2,2,5',
 '0,1,6~2,1,6',
 '1,1,8~1,1,9']

Each row describes the coordinates of a block, starting from lowest z coord to highest.
- format is x1,y1,z1~x2,y2,z2 to describe the volume that block spans

Blocks are supported if there's at least one contact point below, otherwise they fall.

**Part 1: Simulate such that all the blocks settle (some haven't 'landed' down yet) and then find out how many blocks can be removed without things falling down!**

In [53]:
from dataclasses import dataclass

@dataclass
class Brick():
    x: tuple[int, int]
    y: tuple[int, int]
    z: tuple[int, int]

    def can_balance_on_this(self, other_brick: 'Brick'):
        z1s, z1e = self.z
        z2s, z2e = other_brick.z

        return z2s in range(z1s, z1e) or z2e in range(z1s, z1e)

    def fall_vertically_to_row(self, row: int):
        z1, z2 = self.z
        assert row < z1, f"{row=} {z1=}"
        diff = z1 - row
        self.z = (z1-diff, z2-diff)
    
    def overlap_horizontally(self, other_brick: 'Brick'):
        return (
            other_brick.x[0] in range(self.x[0], self.x[1]+1) or
            other_brick.x[1] in range(self.x[0], self.x[1]+1) or
            other_brick.y[0] in range(self.y[0], self.y[1]+1) or
            other_brick.y[1] in range(self.y[0], self.y[1]+1)
        )

In [59]:
class Stack():

    def __init__(self, bricks: list[Brick]):
        self.bricks: list[Brick] = bricks
        bricks.sort(key=lambda b: b.z[0])
        self.shift_bricks_down_as_far_as_they_can()

    
    def shift_bricks_down_as_far_as_they_can(self):
        z_max = max([brick.z[1] for brick in self.bricks])

        def set_flat_rows() -> dict[int, list[Brick]]:
            flat_rows = {z:[] for z in range(1, z_max+1)}
            for brick in self.bricks:
                for z in range(brick.z[0], brick.z[1]+1):
                    flat_rows[z].append(brick)
            return flat_rows
        
        flat_rows = set_flat_rows()
        for brick in self.bricks:
            for z in range(1, brick.z[0]+1):
                if brick.z[0] == z:
                    break
                doesnt_overlap = True
                for sub_brick in flat_rows[z]:
                    doesnt_overlap = not sub_brick.overlap_horizontally(brick)
                    if not doesnt_overlap:
                        break
                if doesnt_overlap:
                    brick.fall_vertically_to_row(z)
                    flat_rows = set_flat_rows()
                    break


In [60]:
def part1(lines: list[str]):
    bricks = []
    for line in lines:
        l, r = line.split("~")
        lx, ly, lz = map(int, l.split(","))
        rx, ry, rz = map(int, r.split(","))
        bricks.append(
            Brick((lx, rx), (ly, ry), (lz, rz))
        )

    stack = Stack(bricks)

    print(stack.bricks)

part1(example_lines)
# part1(input_lines)


[Brick(x=(1, 1), y=(0, 2), z=(1, 1)), Brick(x=(0, 2), y=(0, 0), z=(2, 2)), Brick(x=(0, 2), y=(2, 2), z=(3, 3)), Brick(x=(0, 0), y=(0, 2), z=(4, 4)), Brick(x=(2, 2), y=(0, 2), z=(5, 5)), Brick(x=(0, 2), y=(1, 1), z=(6, 6)), Brick(x=(1, 1), y=(1, 1), z=(7, 8))]


In [61]:
range((1, 2)[0], (1,2)[1])

range(1, 2)

In [ ]:
2 in range(1,2)

False